# Análise dos produtos comercializados

## Criar directoria de apoio 

In [1]:
from pathlib import Path

Path('support').mkdir(parents=True, exist_ok=True)
Path('downloads').mkdir(parents=True, exist_ok=True)

## Criar ficheiro de configuração se não existir

O ficheiro de configuração serve para armazenar
a APIKEY que dá acesso a mais de 500 linhas de
resultados.


In [2]:
import os

fname = 'config.ini'
content = """
# Ficheiro de configuração
[comtrade]
# Add API Key. DO NOT SHARE
key = APIKEYHERE
"""
if not os.path.isfile(fname):
  print("Criando ficheiro de configuração")
  with open(fname,'w') as f:
    f.write(content)


## Obter uma chave de acesso à API

Para aceder à UN Comtrade via API sem limites é necessário uma chave de acesso,
de outro modo os resultados são limitados a 500 linhas.

Para obter a chave de acsso:
* Registo em https://comtradedeveloper.un.org/
* Ir para _Products_ 
* Selecionar "Premium Individual APIs" (https://comtradedeveloper.un.org/product#product=dataapis)
* Escolher _Subscribe to "comtrade - v1"_  
* Esperar pelo email com a chave da API key (demora alguns dias)
* Copiar a chave para o local indicado no ficheiro `config.ini` antes
  de executar o resto do notebook.


In [3]:
import configparser
import comtrade

if os.path.isfile(fname):

    config = configparser.ConfigParser()
    config.read('config.ini')
    APIKEY = config['comtrade']['key']

comtrade.init(APIKEY)

## Parâmetros gerais que não mudam



In [4]:
m49_angola = 24
m49_brazil = 76
m49_cabo_verde = 132
m49_china = 156
m49_hong_kong = 344
m49_macau = 446
m49_guine_equatorial = 226
m49_guine_bissau = 624
m49_mozambique = 508
m49_portugal = 620
m49_stome_principe = 678
m49_timor = 626

# make list of Portuguese Speaking Countries
m49_plp = [m49_angola,m49_brazil,m49_cabo_verde,m49_guine_bissau,
            m49_guine_equatorial,m49_mozambique,m49_portugal,
            m49_stome_principe,m49_timor]
m49_plp_list = ",".join(map(str,m49_plp))




## China, categorias de produtos mais importantes nas trocas com os PLP

Obtem os totais agregados com nível 2 de código HS e lista os primeiros

In [5]:
import comtrade
import pandas as pd


rank_filter = 5  # número de importações mais relevantes
years = "2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021"
flowCode = 'M,X'
pco_cols = ['reporterDesc','partnerDesc','refYear','rank','cmdCode','cmdDesc',
            'flowCode','primaryValue']
df = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flowCode,
                     cmdCode="AG2",
                     reporterCode=m49_china,
                     partnerCode=m49_plp_list,
                     period=years,
                     timeout=120
                     )

pco = df.sort_values(['partnerDesc','refYear','primaryValue'], ascending=[True,True,False])
pco['rank'] = pco.groupby(['partnerDesc','refYear','flowCode'])["primaryValue"].rank(method="dense", ascending=False)
# convert rank column to int
pco['rank'] = pco['rank'].astype(int)

pco_top5 = pco[pco['rank'] <= rank_filter]
cmdCodes_top5 = pco_top5['cmdCode'].unique()
print(f"Categorias de produtos envolvidas: {cmdCodes_top5}")
# save to Excel
pco_top5_sorted = pco_top5[pco_cols].set_index(['reporterDesc','partnerDesc','refYear','flowCode','rank']).sort_index()

# Prepare file name for Excel output
filename_note=f"{years.replace(',','_')}_{flowCode.replace(',','_')}"  # change to append to filename
excel_file_name = f"./downloads/china_plp_top5_{filename_note}.xlsx"
excel_file = pd.ExcelWriter(excel_file_name)
# Save data
pco_top5_sorted.to_excel(excel_file, sheet_name="comtrade", startrow=2)
excel_file.close()
# Write title in first row TBD
excel_file_title = f"Trocas Comerciais China-PLP, top {rank_filter} produtos (M=Importações, X=Exportações), {years}, valores USD"
print(excel_file_title)
print("Guardado em:",excel_file_name)

# show
pd.options.display.max_colwidth=100
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 100
pco_top5_sorted

Categorias de produtos envolvidas: ['27' '64' '07' '85' '62' '34' '44' '73' '39' '63' '03' '25' '88' '52'
 '16' '83' '87' '84' '71' '20' '90' '40' '72' '74' '76' '94' '68' '22'
 '01' '26' '12' '47' '29' '24' '15' '54' '41' '17' '02' '89' '61' '82'
 '95' '55' '10' '70' '60' '09' '69' '99' '92' '49' '38' '30' '58' '48'
 '65' '97' '33' '56' '08' '04' '21' '96' '28' '31' '86' '42' '45' '91'
 '32' '06' '14' '78']
Trocas Comerciais China-PLP, top 5 produtos (M=Importações, X=Exportações), 2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021, valores USD
Guardado em: ./downloads/china_plp_top5_2000_2001_2002_2003_2004_2005_2006_2007_2008_2009_2010_2011_2012_2013_2014_2015_2016_2017_2018_2019_2020_2021_M_X.xlsx


cmdCode  \
reporterDesc partnerDesc refYear flowCode rank           
China        Angola      2000    M        1         27   
                                          2         44   
                                          3         73   
                                          4         39   
                                          5         85   
...                                                ...   
             Timor-Leste 2021    X        1         73   
                                          2         72   
                                          3         39   
                                          4         85   
                                          5         27   

                                                                                                                                            cmdDesc  \
reporterDesc partnerDesc refYear flowCode rank                                                                                                        
China        Angola      2000    M        1     Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral w...   
                                          2                                                                Wood and articles of wood; wood charcoal   
                                          3                                                                                  Iron or steel articles   
                                          4                                                                           Plastics and articles thereof   
                                          5     Electrical machinery and equipment and parts thereof; sound recorders and reproducers; televisio...   
...                                                                                                                                             ...   
             Timor-Leste 2021    X        1                                                                                  Iron or steel articles   
                                          2                                                                                          Iron and steel   
                                          3                                                                           Plastics and articles thereof   
                                          4     Electrical machinery and equipment and parts thereof; sound recorders and reproducers; televisio...   
                                          5     Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral w...   

                                                   primaryValue  
reporterDesc partnerDesc refYear flowCode rank                   
China        Angola      2000    M        1    1,842,016,530.00  
                                          2          671,664.00  
                                          3            4,055.00  
                                          4              600.00  
                                          5               82.00  
...                                                         ...  
             Timor-Leste 2021    X        1       46,230,023.00  
                                          2       22,874,181.00  
                                          3       17,001,590.00  
                                          4       16,813,847.00  
                                          5       16,094,958.00  

[1733 rows x 3 columns]

## Detalhe do tipo de produtos comercializados entre os PLP e a China

Em dois passos: 
* obtém as categorias de nível 2 mais importantes de cada país
* pesquisa todos as subcategorias de cada um das categorias

In [6]:

import ipywidgets as widgets
from IPython.display import display

rank_filter = 5  # número de importações mais relevantes
years = "2021"
partnerCode = m49_angola # 
flowCode="M"

# select year with widget from list range(2000,2022)

years = widgets.SelectMultiple(
    options=range(2000,2022),
    value=[2021],
    description='Ano:',
    disabled=False,
)

# select country widget from list Angola, Brazil, Cabo Verde, Guiné Bissau, Equatorial Guinea, Mozambique, Portugal, São Tomé e Príncipe, Timor-Leste
partnerCodeWidget = widgets.Dropdown(
    options=[("Angola", m49_angola), ("Brazil", m49_brazil), ("Cabo Verde", m49_cabo_verde), ("Guiné Bissau", m49_guine_bissau), ("Guiné Equatorial", m49_guine_equatorial), ("Mozambique", m49_mozambique), ("Portugal", m49_portugal), ("São Tomé e Príncipe", m49_stome_principe), ("Timor-Leste", m49_timor)],
    # value=[("Angola",m49_angola)],
    description='País:',
    disabled=False,
)

# select flowCode widget from list M=Importações, X=Exportações
flowCodeWidget = widgets.Dropdown(
    options=[("Importações", "M"), ("Exportações", "X")],
    value="M",
    description='Tipo:',
    disabled=False, 
)



### Escolher ano, parceiro e fluxo (import/export) para análise 

Pode escolher-se mais que um ano.

In [22]:

display(years)
display(partnerCodeWidget)
display(flowCodeWidget)


SelectMultiple(description='Ano:', index=(21,), options=(2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008,…

Dropdown(description='País:', index=8, options=(('Angola', 24), ('Brazil', 76), ('Cabo Verde', 132), ('Guiné B…

Dropdown(description='Tipo:', options=(('Importações', 'M'), ('Exportações', 'X')), value='M')

Depois de escolher ano, país e fluxo executar a célula seguinte

In [23]:
import comtrade
from comtrade import COUNTRY_CODES, HS_CODES, HS_CODES_DF, HS_CODES_L2_DF


yearsList = ",".join(list(map(str,years.value)))
partnerCode = partnerCodeWidget.value


# decode country
country_name = comtrade.COUNTRY_CODES.get(partnerCode)

flow = flowCodeWidget.value
print(yearsList,country_name,flow)

pco_cols = ['reporterDesc','partnerDesc','refYear','rank','cmdDesc',
            'flowCode','primaryValueFormated']

pco_cols_detail = ['reporterDesc','partnerDesc','refYear','cmdCode','cmdDesc',
            'flowCode','primaryValue', 'isAggregate']

# first we collect the top commodity level 2 codes            
df = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flow,
                     cmdCode="AG2",
                     reporterCode=m49_china,
                     partnerCode=partnerCode,
                     period=yearsList,
                     timeout=30,
                     echo_url=True
                     )

pco = df.sort_values(['partnerDesc','refYear','primaryValue'], ascending=[True,True,False])
pco['rank'] = pco.groupby(['partnerDesc','refYear'])["primaryValue"].rank(method="dense", ascending=False)
pco_top5 = pco[pco['rank'] <= rank_filter]
# get the countries
countries = pco_top5.partnerDesc.unique()
country_cmd_top5_codes = dict()

# get the detailed commodity codes for the top of each country
for country in countries:
    l2_codes = pco_top5[pco_top5.partnerDesc == country]['cmdCode'].unique()
    print(country,l2_codes)
    hs_details = []
    for l2_code in l2_codes:
        l2_sub_codes = list(HS_CODES_DF[HS_CODES_DF.hscode.str.startswith(l2_code)]['hscode'])
        hs_details = hs_details + l2_sub_codes
    # print(hs_details)
    country_cmd_top5_codes[country] = hs_details.copy()

# now we fetch the detail
countryDesc = COUNTRY_CODES[partnerCode]
country_cmd_codes = ",".join(country_cmd_top5_codes[countryDesc])
print(country_cmd_codes)

df = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flow,
                     cmdCode=country_cmd_codes, # sometimes the detail commodity codes generates errors
                     reporterCode=m49_china,
                     partnerCode=partnerCode,
                     partner2Code=0,
                     period=yearsList,
                     echo_url=True,
                     timeout=60
                     )
df[pco_cols_detail].sort_values('primaryValue',ascending=False).to_excel(f"./downloads/china_plp_{yearsList}_{flowCode}_detail_{countryDesc}.xlsx")
print("Mostrando o top 100 geral")
df[df.isAggregate == False][pco_cols_detail].sort_values('primaryValue',ascending=False).head(100)

2021 Timor-Leste M
https://comtradeapi.un.org/data/v1/get//C/A/HS?reporterCode=156&period=2021&partnerCode=626&partner2Code=0&cmdCode=AG2&flowCode=M&customsCode=C00&subscription-key=HIDDEN
Timor-Leste ['27' '09' '14' '85' '44']
27,2701,270111,270112,270119,270120,2702,270210,270220,2703,270300,2704,270400,2705,270500,2706,270600,2707,270710,270720,270730,270740,270750,270791,270799,2708,270810,270820,2709,270900,2710,271012,271019,271020,271091,271099,2711,271111,271112,271113,271114,271119,271121,271129,2712,271210,271220,271290,2713,271311,271312,271320,271390,2714,271410,271490,2715,271500,2716,271600,09,0901,090111,090112,090121,090122,090190,0902,090210,090220,090230,090240,0903,090300,0904,090411,090412,090421,090422,0905,090510,090520,0906,090611,090619,090620,0907,090710,090720,0908,090811,090812,090821,090822,090831,090832,0909,090921,090922,090931,090932,090961,090962,0910,091011,091012,091020,091030,091091,091099,14,1401,140110,140120,140190,1404,140420,140490,85,8501,850110

/workspaces/cipf-comtrade/comtrade.py:229: UserWarning: Server returned HTTP Status: 404
  warnings.warn(f"Server returned HTTP Status: "+str(resp.status_code),)
/workspaces/cipf-comtrade/comtrade.py:230: UserWarning: b'The resource you are looking for has been removed, had its name changed, or is temporarily unavailable.'
  warnings.warn(str(resp.content))


TypeError: 'NoneType' object is not subscriptable